## Classical Notebook

`Importing libraries`

In [24]:
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
import folium
import itertools
from geopy.distance import geodesic
import time
from typing import List, Dict, Tuple
import matplotlib.pyplot as plt
import requests
import json

ox.settings.use_cache = True
ox.settings.log_console = True

 `Define hospital and patient data`


In [ ]:
data = {
    "hospital": {
        "name": "Central Hospital",
        "latitude": 29.99512653425452,
        "longitude": 31.68462840171934,
        "type": "destination"
    },
    "patients": [
        {"id": "DT", "name": "Patient DT", "latitude": 30.000417586266437, "longitude": 31.73960813272627},
        {"id": "GR", "name": "Patient GR", "latitude": 30.011344405285193, "longitude": 31.747827362371993},
        {"id": "R2", "name": "Patient R2", "latitude": 30.030388325206854, "longitude": 31.669231198639675},
        {"id": "R3_2", "name": "Patient R3_2", "latitude": 30.030940768851426, "longitude": 31.688371339937028},
        {"id": "IT", "name": "Patient IT", "latitude": 30.01285635906825, "longitude": 31.693811715848444}
    ]
}

`Convert hospital and patient data into DataFrames`

In [26]:
hospital_df = pd.DataFrame([data["hospital"]])
patients_df = pd.DataFrame(data["patients"])
patients_df["type"] = "patient"
locations_df = pd.concat([hospital_df, patients_df], ignore_index=True)
locations_df.drop_duplicates(subset=["latitude", "longitude"], inplace=True)
locations_df.reset_index(drop=True, inplace=True)

print("Locations DataFrame:")
locations_df

Locations DataFrame:


,name,latitude,longitude,type,id
0,Central Hospital,29.995127,31.684628,destination,NaN
1,Patient DT,30.000418,31.739608,patient,DT
2,Patient GR,30.011344,31.747827,patient,GR
3,Patient R2,30.030388,31.669231,patient,R2
4,Patient R3_2,30.030941,31.688371,patient,R3_2
5,Patient IT,30.012856,31.693812,patient,IT


`Calculate Open Source Routing Machine (OSRM) distances between hospital and patients`

In [27]:
#Function to calculate OSRM distances between hospital and patients
def calculate_osrm_distances(locations_df):
    coordinates = []
    location_names = []

    hospital = locations_df[locations_df['type'] == 'destination'].iloc[0]
    coordinates.append(f"{hospital['longitude']},{hospital['latitude']}")
    location_names.append(hospital['name'])

    for idx, row in locations_df[locations_df['type'] == 'patient'].iterrows():
        coordinates.append(f"{row['longitude']},{row['latitude']}")
        location_names.append(row['name'])

    coords_str = ';'.join(coordinates)
    url = f"http://router.project-osrm.org/table/v1/driving/{coords_str}?annotations=distance"

    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()

        if 'distances' in data:
            distance_matrix = {}

            for i, loc1 in enumerate(location_names):
                distance_matrix[loc1] = {}
                for j, loc2 in enumerate(location_names):
                    distance_meters = data['distances'][i][j]
                    distance_km = distance_meters / 1000 if distance_meters is not None else float('inf')
                    distance_matrix[loc1][loc2] = distance_km

            return distance_matrix
        else:
            print("OSRM response format error")
            return None

    except requests.exceptions.RequestException as e:
        print(f"OSRM API error: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        return None


 `Build and display the distance matrix`

In [28]:
distance_matrix = calculate_osrm_distances(locations_df)

if distance_matrix:
    print("Real road distance matrix in km:")
    distance_df = pd.DataFrame(distance_matrix)
    print(distance_df.round(2))
else:
    print("Falling back to haversine distance...")
    distance_matrix = {}
    for i, row1 in locations_df.iterrows():
        distance_matrix[row1['name']] = {}
        for j, row2 in locations_df.iterrows():
            if i == j:
                distance_matrix[row1['name']][row2['name']] = 0
            else:
                dist = geodesic((row1['latitude'], row1['longitude']),
                               (row2['latitude'], row2['longitude'])).km
                distance_matrix[row1['name']][row2['name']] = dist
    print("Haversine distance matrix (km):")
    distance_df = pd.DataFrame(distance_matrix)
    print(distance_df.round(2))

Real road distance matrix in km:
                  Central Hospital  Patient DT  Patient GR  Patient R2  \
Central Hospital              0.00       14.19       17.78       11.86   
Patient DT                    8.63        0.00        7.75       19.67   
Patient GR                   11.50        2.36        0.00       15.66   
Patient R2                    9.45       10.92       11.81        0.00   
Patient R3_2                 10.85        9.24       10.12       11.57   
Patient IT                    9.67        9.43       10.48       11.54   

                  Patient R3_2  Patient IT  
Central Hospital          7.34        9.27  
Patient DT               12.17        9.40  
Patient GR               10.05       12.27  
Patient R2                4.07        7.32  
Patient R3_2              0.00        8.72  
Patient IT                5.93        0.00  


`Route Optimization Class (AmbulanceRouter)`

In [ ]:
#Route Optimization Class
class AmbulanceRouter:

#Initialize the AmbulanceRouter
    def __init__(self, distance_matrix, hospital_name, max_stops=3):
        self.distance_matrix = distance_matrix
        self.hospital_name = hospital_name
        self.max_stops = max_stops
        self.patient_names = [name for name in distance_matrix.keys() if name != hospital_name]

#Calculate the total round-trip distance for a given trip
    def calculate_trip_distance(self, trip):
        if not trip:
            return 0

        total_distance = self.distance_matrix[self.hospital_name][trip[0]]

        for i in range(len(trip) - 1):
            total_distance += self.distance_matrix[trip[i]][trip[i+1]]

        total_distance += self.distance_matrix[trip[-1]][self.hospital_name]
        return total_distance

#Generate all possible valid trips of size 1 up to max stops patients
    def generate_all_possible_trips(self):
        all_trips = []
        for num_stops in range(1, self.max_stops + 1):
            for combo in itertools.combinations(self.patient_names, num_stops):
                for perm in itertools.permutations(combo):
                    all_trips.append(list(perm))
        return all_trips

#Bruteforce method to find the optimal routing strategy
    def brute_force_optimization(self):
        all_trips = self.generate_all_possible_trips()
        best_total_distance = float('inf')
        best_routes = []

        for num_trips in range(1, len(self.patient_names) + 1):
            for trip_combination in itertools.combinations(all_trips, num_trips):
                covered_patients = set()
                for trip in trip_combination:
                    covered_patients.update(trip)

                if covered_patients == set(self.patient_names):
                    total_distance = sum(self.calculate_trip_distance(trip) for trip in trip_combination)
                    if total_distance < best_total_distance:
                        best_total_distance = total_distance
                        best_routes = list(trip_combination)
        return best_routes, best_total_distance

#Greedy heuristic to quickly build routes (not guaranteed optimal)
    def greedy(self):
        remaining_patients = set(self.patient_names)
        routes = []
        total_distance = 0

        while remaining_patients:
            current_location = self.hospital_name
            current_trip = []
            current_distance = 0

            for _ in range(self.max_stops):
                if not remaining_patients:
                    break

                closest_patient = None
                min_distance = float('inf')

                for patient in remaining_patients:
                    dist = self.distance_matrix[current_location][patient]
                    if dist < min_distance:
                        min_distance = dist
                        closest_patient = patient

                if closest_patient:
                    current_trip.append(closest_patient)
                    current_distance += min_distance
                    current_location = closest_patient
                    remaining_patients.remove(closest_patient)

            if current_trip:
                current_distance += self.distance_matrix[current_location][self.hospital_name]
                routes.append(current_trip)
                total_distance += current_distance
        return routes, total_distance

`Greedy Optimization and Display Results`

In [ ]:
router = AmbulanceRouter(distance_matrix, "Central Hospital")
optimal_routes, total_distance = router.greedy() # optimal routes using greedy heuristic

print(f"\nOptimal Routes (Total Distance: {total_distance:.2f} km):")
for i, route in enumerate(optimal_routes, 1):
    print(f"Trip {i}: Hospital -> {' -> '.join(route)} -> Hospital (Distance: {router.calculate_trip_distance(route):.2f} km)")


Optimal Routes (Total Distance: 58.71 km):
Trip 1: Hospital -> Patient DT -> Patient GR -> Patient R3_2 -> Hospital (Distance: 28.46 km)
Trip 2: Hospital -> Patient R2 -> Patient IT -> Hospital (Distance: 30.25 km)


`Get OSRM Routes for Visualization`

In [31]:
#Get OSRM Routes for Visualization
def get_osrm_route(coords):
    coords_str = ';'.join([f"{lon},{lat}" for lat, lon in coords])
    url = f"http://router.project-osrm.org/route/v1/driving/{coords_str}?overview=full&geometries=geojson"

    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()

        if data['code'] == 'Ok' and data['routes']:
            return data['routes'][0]['geometry']['coordinates']
    except:
        pass

    return None

def create_interactive_map(locations_df, optimal_routes, hospital_name):
    hospital = locations_df[locations_df['name'] == hospital_name].iloc[0]
    m = folium.Map(location=[hospital['latitude'], hospital['longitude']], zoom_start=13)

    folium.Marker(
        [hospital['latitude'], hospital['longitude']],
        popup=hospital_name,
        icon=folium.Icon(color='red', icon='hospital-o', prefix='fa')
    ).add_to(m)

    for idx, row in locations_df[locations_df['type'] == 'patient'].iterrows():
        folium.Marker(
            [row['latitude'], row['longitude']],
            popup=row['name'],
            icon=folium.Icon(color='blue', icon='user-md', prefix='fa')
        ).add_to(m)

    colors = ['green', 'purple', 'orange', 'darkred', 'lightblue']

    for i, route in enumerate(optimal_routes):
        if not route:
            continue
        route_coords = []

        hospital_coord = (hospital['latitude'], hospital['longitude'])
        first_patient = locations_df[locations_df['name'] == route[0]].iloc[0]
        first_coord = (first_patient['latitude'], first_patient['longitude'])

        segment = get_osrm_route([hospital_coord, first_coord])
        if segment:
            route_coords.extend([(lat, lon) for lon, lat in segment])


        for j in range(len(route) - 1):
            patient1 = locations_df[locations_df['name'] == route[j]].iloc[0]
            patient2 = locations_df[locations_df['name'] == route[j+1]].iloc[0]
            coord1 = (patient1['latitude'], patient1['longitude'])
            coord2 = (patient2['latitude'], patient2['longitude'])

            segment = get_osrm_route([coord1, coord2])
            if segment:
                route_coords.extend([(lat, lon) for lon, lat in segment][1:])

        last_patient = locations_df[locations_df['name'] == route[-1]].iloc[0]
        last_coord = (last_patient['latitude'], last_patient['longitude'])

        segment = get_osrm_route([last_coord, hospital_coord])
        if segment:
            route_coords.extend([(lat, lon) for lon, lat in segment][1:])

        if route_coords:
            folium.PolyLine(
                route_coords,
                color=colors[i % len(colors)],
                weight=5,
                opacity=0.7,
                popup=f"Trip {i+1}: {' -> '.join(route)}"
            ).add_to(m)

    return m


`Create interactive map for Greedy routes`

In [ ]:
route_map = create_interactive_map(locations_df, optimal_routes, "Central Hospital") # Create interactive map for Greedy routes
route_map.save('ambulance_routes_real_roads.html')
route_map

`Brute Force Optimization and Visualization`

In [40]:
def run_brute_force_optimization():
    start_time = time.time()
    brute_force_routes, brute_force_distance = router.brute_force_optimization()
    brute_force_time = time.time() - start_time

    print(f"\nBrute Force Optimal Routes (Total Distance: {brute_force_distance:.2f} km):")
    print(f"Computation Time: {brute_force_time:.2f} seconds")

    for i, route in enumerate(brute_force_routes, 1):
        route_distance = router.calculate_trip_distance(route)
        print(f"Trip {i}: Hospital -> {' -> '.join(route)} -> Hospital (Distance: {route_distance:.2f} km)")

    return brute_force_routes, brute_force_distance, brute_force_time

brute_force_routes, brute_force_distance, brute_force_time = run_brute_force_optimization()

greedy_routes, greedy_distance = router.optimized_greedy()
improvement = ((greedy_distance - brute_force_distance) / brute_force_distance) * 100

print(f"\nComparison:")
print(f"Greedy Algorithm: {greedy_distance:.2f} km")
print(f"Brute Force (Optimal): {brute_force_distance:.2f} km")
print(f"Greedy is {improvement:.2f}% {'worse' if improvement > 0 else 'better'} than optimal")

brute_force_map = create_interactive_map(locations_df, brute_force_routes, "Central Hospital")
brute_force_map.save('ambulance_routes_brute_force.html')

brute_force_map


Brute Force Optimal Routes (Total Distance: 57.31 km):
Computation Time: 151.06 seconds
Trip 1: Hospital -> Patient IT -> Patient R2 -> Hospital (Distance: 28.85 km)
Trip 2: Hospital -> Patient DT -> Patient GR -> Patient R3_2 -> Hospital (Distance: 28.46 km)

Comparison:
Greedy Algorithm: 58.71 km
Brute Force (Optimal): 57.31 km
Greedy is 2.44% worse than optimal


` Detailed Route Analysis`

In [36]:
def analyze_routes(locations_df, distance_matrix, optimal_routes, hospital_name):
    print("\n" + "="*60)
    print("DETAILED ROUTE ANALYSIS")
    print("="*60)

    total_distance = 0
    for i, route in enumerate(optimal_routes, 1):
        trip_distance = 0
        print(f"\nTrip {i}: Hospital -> {' -> '.join(route)} -> Hospital")

        # Hospital to first patient
        dist1 = distance_matrix[hospital_name][route[0]]
        trip_distance += dist1
        print(f"  Hospital → {route[0]}: {dist1:.2f} km")

        # Between patients
        for j in range(len(route) - 1):
            dist = distance_matrix[route[j]][route[j+1]]
            trip_distance += dist
            print(f"  {route[j]} → {route[j+1]}: {dist:.2f} km")

        # Last patient to hospital
        dist2 = distance_matrix[route[-1]][hospital_name]
        trip_distance += dist2
        print(f"  {route[-1]} → Hospital: {dist2:.2f} km")

        print(f"  Total trip distance: {trip_distance:.2f} km")
        total_distance += trip_distance

    print(f"\nOverall total distance: {total_distance:.2f} km")
    print(f"Number of trips: {len(optimal_routes)}")
    print(f"Average patients per trip: {len(locations_df[locations_df['type'] == 'patient']) / len(optimal_routes):.2f}")

    analyze_routes(locations_df, distance_matrix, optimal_routes, "Central Hospital")


`Performance Comparison`

In [37]:
def compare_algorithms():
    print("\n" + "="*60)
    print("ALGORITHM COMPARISON")
    print("="*60)

    # Greedy optimized
    start_time = time.time()
    routes_greedy, dist_greedy = router.optimized_greedy()
    time_greedy = time.time() - start_time

    print(f"Optimized Greedy:")
    print(f"  Distance: {dist_greedy:.2f} km")
    print(f"  Time: {time_greedy:.4f} seconds")
    print(f"  Trips: {len(routes_greedy)}")

    # brute force for small instances
    if len(router.patient_names) <= 5:
        try:
            start_time = time.time()
            routes_brute, dist_brute = router.brute_force_optimization()
            time_brute = time.time() - start_time

            print(f"\nBrute Force (Optimal):")
            print(f"  Distance: {dist_brute:.2f} km")
            print(f"  Time: {time_brute:.4f} seconds")
            print(f"  Trips: {len(routes_brute)}")

            # Show improvement
            improvement = ((dist_greedy - dist_brute) / dist_brute) * 100
            print(f"\nGreedy is {improvement:.2f}% worse than optimal")

        except Exception as e:
            print(f"\nBrute force failed: {e}")

compare_algorithms()


ALGORITHM COMPARISON
Optimized Greedy:
  Distance: 58.71 km
  Time: 0.0000 seconds
  Trips: 2

Brute Force (Optimal):
  Distance: 57.31 km
  Time: 151.1041 seconds
  Trips: 2

Greedy is 2.44% worse than optimal


`Export Results`

In [38]:
def export_results(locations_df, distance_matrix, optimal_routes, total_distance):
    results = []

    for i, route in enumerate(optimal_routes, 1):
        trip_distance = router.calculate_trip_distance(route)
        results.append({
            'trip_number': i,
            'route': ' -> '.join(['Hospital'] + route + ['Hospital']),
            'distance_km': round(trip_distance, 2),
            'patients_served': ', '.join(route),
            'number_of_stops': len(route)
        })

    results_df = pd.DataFrame(results)
    results_df.to_csv('ambulance_routing_results.csv', index=False)

    # Export distance matrix
    distance_df = pd.DataFrame(distance_matrix)
    distance_df.to_csv('distance_matrix.csv')
    return results_df

# Export results
results_df = export_results(locations_df, distance_matrix, optimal_routes, total_distance)
print("\nExported Results:")
print(results_df)


Exported Results:
   trip_number                                              route  \
0            1  Hospital -> Patient DT -> Patient GR -> Patien...   
1            2   Hospital -> Patient R2 -> Patient IT -> Hospital   

   distance_km                       patients_served  number_of_stops  
0        28.46  Patient DT, Patient GR, Patient R3_2                3  
1        30.25                Patient R2, Patient IT                2  
